<a href="https://colab.research.google.com/github/Riya-Phagna/Quantitative-Trading-Strategy-NIFTY-Regime-Based-System/blob/main/NIFTY_Regime_Based_Quantitative_Trading_Systemipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DATA UTILS

In [1]:
import pandas as pd
import numpy as np

def load_csv(path):
    df = pd.read_csv(path)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.set_index('timestamp', inplace=True)
    return df

def align_timeframes(*dfs):
    common_index = dfs[0].index
    for df in dfs[1:]:
        common_index = common_index.intersection(df.index)
    return [df.loc[common_index] for df in dfs]

def winsorize(series, lower=0.001, upper=0.999):
    return series.clip(series.quantile(lower), series.quantile(upper))


GREEKS

In [10]:
!pip install py_vollib


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for py_vollib: filename=py_vollib-1.0.1-py3-none-any.whl size=62833 sha256=de995cf889b5b48fdcb4c8b2e38b35cf5e55e7f840ed27ea9fc540ac0f79c9c4
  Stored in directory: /root/.cache/pip/wheels/5f/8c/ab/bbc3a2b0e6ae83633bb99f960459a2ab960b4672c7440ff4ef
  Created wheel for py_lets_be_rational: filename=py_lets_be_rational-1.0.1-py3-none-any.whl size=24451 sha256=426b965e172b43eb9e8f82e3c7349f6ce3b2ed0f5e4e812e665677635c0462c1
  Stored in directory: /root/.cache/pip/wheels/c3/c6/50/415aabc2033e7651cc8cc590bd01c5f23f9050328f6016d6a5
Successfully built py_vollib py_lets_be_rational


In [11]:
from py_vollib.black_scholes.greeks.analytical import delta, gamma, theta, vega, rho

def compute_greeks(flag, S, K, T, r, iv):
    return {
        "delta": delta(flag, S, K, T, r, iv),
        "gamma": gamma(flag, S, K, T, r, iv),
        "theta": theta(flag, S, K, T, r, iv),
        "vega": vega(flag, S, K, T, r, iv),
        "rho": rho(flag, S, K, T, r, iv)
    }


FEATURES

In [14]:
!pip install ta


  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=d5fbbfd16372b68b07f15dfd387392d9b930d75c4c99f3e28e96a9aaf78ac624
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta


REGIME

In [17]:
import pandas as pd
from ta.trend import EMAIndicator

def add_ema(df):
    df['ema_5'] = EMAIndicator(close=df['close'], window=5).ema_indicator()
    df['ema_15'] = EMAIndicator(close=df['close'], window=15).ema_indicator()
    return df

def derived_features(df):
    df['avg_iv'] = (df['call_iv'] + df['put_iv']) / 2
    df['iv_spread'] = df['call_iv'] - df['put_iv']
    df['pcr_oi'] = df['put_oi'] / df['call_oi']
    df['pcr_vol'] = df['put_vol'] / df['call_vol']
    df['fut_basis'] = (df['fut_close'] - df['spot_close']) / df['spot_close']
    df['spot_ret'] = df['spot_close'].pct_change()
    return df


In [15]:
!pip install hmmlearn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 kB 12.1 MB/s eta 0:00:00


In [16]:
import numpy as np
from hmmlearn.hmm import GaussianHMM

def train_hmm(X, n_states=3):
    model = GaussianHMM(
        n_components=n_states,
        covariance_type="full",
        n_iter=1000
    )
    model.fit(X)
    return model

def predict_regime(model, X):
    return model.predict(X)


BACKTEST

In [7]:
import numpy as np

def backtest(df):
    df['position'] = df['signal'].shift(1)
    df['returns'] = df['close'].pct_change()
    df['strategy_ret'] = df['position'] * df['returns']

    equity = (1 + df['strategy_ret']).cumprod()
    return equity, df


ML MODULES

In [6]:
from xgboost import XGBClassifier

def train_xgboost(X_train, y_train):
    model = XGBClassifier(
        n_estimators=300,
        max_depth=5,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8
    )
    model.fit(X_train, y_train)
    return model
